In [ ]:
# !pip install onnxruntime

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy # install command: pip install spacy
import string
import nltk # install command: pip install nltk
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
import onnxruntime # install command: pip install onnxruntime
import torch.onnx as onnx

In [ ]:
# !python3 -m spacy download en_core_web_sm

In [ ]:
# nltk.download('wordnet')

In [ ]:
# nltk.download('omw-1.4')

In [ ]:
config = {
    'max_features': 1000,
    'num_epochs':200,
    'learning_rate':1e-1,
    'batch_size':64,
    'train_percentage':90
}

# Dataset

Note that the dataset is taken from https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset






Exploration

In [ ]:
df = pd.read_csv('/content/Tweets.csv')
df.head()

In [ ]:
df['sentiment'].unique()

In [ ]:
df = df.loc[df['sentiment']!='neutral']

In [ ]:
df =df[['text','sentiment']]
df.head()

In [ ]:
df['text'].count()

# Preprocessing



# Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatizer.lemmatize("settings")

# Stopword Removal

In [ ]:
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

In [ ]:
type(stopwords)

In [ ]:
len(stopwords)

In [ ]:
list(stopwords)[:10]

# Preprocessing Pipeline

In [ ]:
def preprocess(txt):
    text = txt.lower()
    tokens = txt.split()
    tokens = [lemmatizer.lemmatize(token)for token in tokens]
    txt = ''.join(tokens)
    txt =txt.translate(str.maketrans('','',string.punctuation))
    tokens = txt.split()
    tokens=[token for token in tokens if token not in stopwords]
    txt = ''.join(tokens)
    txt =re.sub('r[0-9]+','',txt)
    return txt

In [ ]:
df = df.reset_index()  # restore the indices of the dataframe so that it star

In [ ]:
original_txt = df['text'][50]
processed_txt = preprocess(df['text'][50])
print(f'The original text was:\n{original_txt}\n The preprocessed txt is :\n{processed_txt} ')

In [ ]:
df['preprocessed_text'] = df['text'].apply(lambda x: preprocess(str(x)))
df.head()

In [ ]:
texts = df['preprocessed_text'].tolist()

In [ ]:
# vectorizer = CountVectorizer(max_features=config['max_features'])
# features = vectorizer.fit_transform(texts)
vectorizer =TfidfVectorizer(max_features=config['max_features'])
features = vectorizer.fit_transform(texts)

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
np_features= features.toarray()

In [ ]:
np_features.shape

We should also work with the labels...

In [ ]:
df['num_sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
df.head()


In [ ]:
labels = df['num_sentiment'].tolist()

In [ ]:
labels[:5]

# Train / Test / Dev Split

In [ ]:
f_train, f_rem, l_train, l_rem = train_test_split(np_features, labels, test_size=1-config['train_percentage']/100, random_state=50)
f_test, f_dev, l_test, l_dev = train_test_split(f_rem, l_rem, test_size=0.5,random_state=50)

In [ ]:
print(f'train features: {f_train.shape}, dev features: {f_dev.shape}, test features :{f_test.shape}')

In [ ]:
print(f'train labels: {len(l_train)}, dev labels: {len(l_dev)}, test labels: {len(l_test)}')

# Converting Everything to Tensors

The numpy array we defined above should be converted to a tensor. This tensor will be used in a "Dataset" object.

In [ ]:
class MyVectorDataset(Dataset):
    def __init__(self,features,labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return  self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]),torch.Tensor(self.labels[idx])
    
          
        

In [ ]:
train_dataset=MyVectorDataset(f_train, l_train)
test_dataset=MyVectorDataset(f_test,l_test)
dev_dataset=MyVectorDataset(f_dev, l_dev)

In [ ]:
train_dataloader= DataLoader(train_dataset,batch_size=config['batch_size'],shuffle=True)
test_dataloader= DataLoader(test_dataset,batch_size=config['batch_size'],shuffle=True)
dev_dataloader= DataLoader(dev_dataset,batch_size=config['batch_size'],shuffle=True)


# Neural Net Architecture

In [ ]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
class my_neural_net(torch.nn.Module):
    def __init__(self):
        super(my_neural_net, self).__init__() 
        self.first_layer = torch.nn.Sequential( 
            nn.Linear(config['max_features'],1),
            nn.Sigmoid()    
        )
    def forward(self, x):
        output = self.first_layer(x)
        return output   

In [ ]:
simple_nn = my_neural_net()

In [ ]:
simple_nn = simple_nn.to(device)

In [ ]:
simple_nn(train_dataset[:2][0])

In [ ]:
simple_nn(train_dataset[:2][0]).shape

# Training


# Binary Cross-Entropy

In [ ]:
loss_fn = nn.BCELoss()

# Optimizer
Note that stochastic gradient descent performs a parameter update for each training example Xi and yi label 



In [ ]:
optimizer = torch.optim.SGD(simple_nn.parameters(),lr=config['learning_rate'])

In [ ]:
def output_to_label(out):
    dis_to_0 = abs(out)
    dis_to_1 = abs(out-1)
    if dis_to_0 <= dis_to_1:
        return 0
    else:
        return 1

In [ ]:
def train_loop(dataloader,
    model,loss_fn,Optimizer,epoch_num):
    num_points= len(dataloader.dataset)
    for batch,(features,labels) in enumerate(dataloader):
        #compute predicton and loss
        pred = model(features)
        loss = loss_fn(pred,labels)
        #Backpropqgeation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_t
        if batch %100 == 0:
             loss, current = loss.item(), batch * len(features)
             print(f"\r Epoch{epoch_num}_loss:{loss:7f}[{current:>5d}/{num_points:>5d}]",end="")

def test_loop(dataloader,model,loss_fn,epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss,correct=0,0
    with torch.no_grad():
       with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.numpy().squeeze()) 
            match_lst=[1 if p==o else 0 for (p,o) in zip(pred_lst,output_lst)]
    sum_test_loss /= num_points
    correct /= num_points   
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")


In [ ]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, simple_nn, loss_fn, optimizer, epoch_num)
    test_loop(dev_dataloader, simple_nn, loss_fn, epoch_num, 'Development/Validation')

# saving the model 

In [ ]:
torch.save(simple_nn.state_dict(),"neural_net.path")

# Load the model

In [ ]:
model = my_neural_net()
model.load_state_dict(torch.load("neural_net.path"))
model.eval()

In [ ]:
model(test_dataset[:2][0])

In [ ]:
l_test[:2]

# The ONNX Format

This format is useful when you want to use your model while coding in Java, Javascript, and C#!

# Save the Model

In [ ]:
dummy_input = torch.zeros((1,config['max_features']))

In [ ]:
onnx.export(model,dummy_input,'neural_net.onnx')

# Inference

In [ ]:
session = onnxruntime.InferenceSession('neural_net.onnx',None)

In [ ]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [ ]:
input_name

In [ ]:
output_name

In [ ]:
result = session.run([output_name], {input_name: test_dataset[0][0].numpy().reshape(1,-1)})

In [ ]:
result